In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Analysis

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
train_df = pd.read_csv("/kaggle/input/pubg-finish-placement-prediction/train_V2.csv")

In [ ]:
target = 'winPlacePerc'

In [ ]:
train_df

In [ ]:
train_df.columns

In [ ]:
train_df.corr()[target]

In [ ]:
train_df_wp = train_df.drop(train_df[train_df.winPoints == 0].index)

In [ ]:
np.corrcoef(train_df_wp.winPoints, train_df_wp[target])

In [ ]:
train_df.isna().sum()

In [ ]:
train_df.winPlacePerc.fillna(-1, inplace = True)

In [ ]:
train_df.matchType.value_counts()

In [ ]:
train_df[train_df.matchType == "squad"].winPlacePerc.mean()

In [ ]:
train_df[train_df.matchType == "duo"].winPlacePerc.mean()

In [ ]:
train_df[train_df.matchType == "solo"].winPlacePerc.mean()

In [ ]:
kills_sorted = train_df[train_df.matchType == "solo"][:100000].sort_values("kills")

plt.scatter(kills_sorted[target], kills_sorted.kills)

In [ ]:
def onehot(series, name):
    series_new = pd.DataFrame()
    for col in series.unique():
        series_new[name + "_{0}".format(col)] = series == col
    
    return (series_new.astype(int), series_new.columns)

In [ ]:
train_df.matchType.value_counts()

In [ ]:
encoded_ = onehot(train_df.matchType, "matchType")

In [ ]:
train_df_j = train_df.join(encoded_[0])

train_df_j = train_df_j.drop('matchType', axis = 1)

train_df_j

In [ ]:
cat_features_encoded = list(encoded_[1])

cat_features_encoded

# Train

In [ ]:
from catboost import CatBoostRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

random_seed = 800

In [ ]:
features = train_df_j.columns

features = features.drop(['Id', 'groupId', 'matchId', target])
features

In [ ]:
XTrain, XTest, YTrain, YTest = train_test_split(train_df_j[features], train_df_j[target], test_size = 0.33, random_state = random_seed)

In [ ]:
model = CatBoostRegressor(
                        random_state = random_seed,
                        n_estimators = 1000,
                        depth = 8,
                        cat_features = cat_features_encoded,
                        loss_function = "MAE")

In [ ]:
model.fit(XTrain, YTrain)

In [ ]:
prediction = model.predict(XTest)

prediction[prediction < 0] = 0
prediction[prediction > 1] = 1

In [ ]:
np.isnan(prediction).sum()

In [ ]:
np.isnan(YTest).sum()

In [ ]:
mean_absolute_error(prediction, YTest)

# Prediction

In [ ]:
test_df = pd.read_csv("/kaggle/input/pubg-finish-placement-prediction/test_V2.csv")
test_df

In [ ]:
test_df

In [ ]:
encoded_ = onehot(test_df.matchType, "matchType")

In [ ]:
test_df_j = test_df.copy()

train_df_j = test_df_j.drop('matchType', axis = 1)

test_df_j = test_df_j.join(encoded_[0])

In [ ]:
features

In [ ]:
test_df_j.columns

In [ ]:
out_df = pd.DataFrame(np.array([test_df_j.Id.values, model.predict(test_df_j[features])]).T, columns = ['Id', 'winPlacePerc'])

In [ ]:
out_df

In [ ]:
out_df.to_csv('submission.csv', index = False)